In [1]:
import pandas as pd
import numpy as np
import openpyxl 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

In [3]:
xls = pd.ExcelFile('youtube_data.xlsx')
df_channel = pd.read_excel(xls, 'channel_data')
df_video = pd.read_excel(xls, 'video_data_complete')

# Video_Data

## infos df videos

In [ ]:
df_video.head(3)

In [ ]:
df_video.info()

## analyse number and time of videos

### df nb video/lenght over the year

In [5]:
df_video_filter = df_video[df_video['duration'].notnull()]

In [7]:
df_nbvideo = pd.DataFrame(data=df_video_filter.groupby('year').size())
df_nbvideo=df_nbvideo.reset_index()

df_nbvideo = df_nbvideo.rename({0: 'nbVideo_Tot'},axis=1)
df_nbvideo['nbVideo_Day']= np.round((df_nbvideo['nbVideo_Tot']/365),1)
df_nbvideo

,year,nbVideo_Tot,nbVideo_Day
0,2018,9045,24.8
1,2019,793,2.2
2,2020,1048,2.9
3,2021,6097,16.7
4,2022,23105,63.3
5,2023,27304,74.8


In [9]:
df_duration = pd.DataFrame(data=df_video_filter.groupby('year')['duration'].agg(['sum','mean']))
df_duration=df_duration.reset_index()

df_vid_time = pd.merge(df_duration,df_nbvideo,how='inner',on='year')
df_vid_time = df_vid_time.rename({'sum': 'sum_duration'},axis=1)
df_vid_time = df_vid_time.rename({'mean': 'mean_duration'},axis=1)

In [11]:
count_shorts = pd.DataFrame((df_video_filter[df_video_filter['duration']<=0.59])['year'].value_counts())
count_s = pd.DataFrame((df_video_filter[(df_video_filter['duration']>0.59)&(df_video_filter['duration']<=5)])['year'].value_counts())
count_m = pd.DataFrame((df_video_filter[(df_video_filter['duration']>5)&(df_video_filter['duration']<=20)])['year'].value_counts())
count_l = pd.DataFrame((df_video_filter[(df_video_filter['duration']>20)&(df_video_filter['duration']<=45)])['year'].value_counts())
count_xl = pd.DataFrame((df_video_filter[df_video_filter['duration']>45])['year'].value_counts())

In [13]:
count = pd.concat([count_shorts, count_s, count_m, count_l, count_xl],axis=1)
name = ['count_shorts', 'count_s', 'count_m', 'count_l', 'count_xl']
count.columns = name
count = count.reset_index()
count = count.fillna(0)

In [15]:
df_vid_time = pd.merge(df_vid_time, count,on='year',how='inner')
df_vid_time

,year,sum_duration,mean_duration,nbVideo_Tot,nbVideo_Day,count_shorts,count_s,count_m,count_l,count_xl
0,2018,31937.61,3.530968,9045,24.8,19,8778,242,5,1
1,2019,3072.57,3.874615,793,2.2,1,738,48,4,2
2,2020,7407.52,7.068244,1048,2.9,7,771,192,68,10
3,2021,64640.21,10.601970,6097,16.7,1,4414,997,324,361
4,2022,107886.81,4.669414,23105,63.3,23,21065,1237,657,123
5,2023,123930.01,4.538896,27304,74.8,11683,13052,1048,1094,427


In [19]:
df_vid_time.to_excel('year_time.xlsx')

### analyse time and number over the year

In [ ]:
fig,axes = plt.subplots(nrows=1, ncols=2,figsize=(12,5))


axes[0].plot(df_vid_time['year'], df_vid_time['nbVideo_Tot'])
axes[0].bar(df_vid_time['year'], df_vid_time['count_shorts'],label='youtube shorts')
axes[0].bar(df_vid_time['year'],df_vid_time['count_s'], bottom = df_vid_time['count_shorts'],label='videos under 5 min')
axes[0].bar(df_vid_time['year'],df_vid_time['count_m'], bottom = df_vid_time['count_shorts'],label='videos betwwen 5 to 20min')
axes[0].bar(df_vid_time['year'],df_vid_time['count_l'], bottom = df_vid_time['count_shorts'],label='videos between 20 to 45min')
axes[0].bar(df_vid_time['year'],df_vid_time['count_xl'], bottom = df_vid_time['count_shorts'],label='videos longer than 45min')
axes[0].set_title('Number of videos over the years')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number of videos')
axes[0].set_ylim(0,28000)


axes[0].legend()

axes[1].plot(df_vid_time['year'], df_vid_time['sum_duration'])
axes[1].set_title('Watch time over the years')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Duration of videos')
axes[1].set_ylim(0,130000)


In [ ]:
short_video = df_video_filter[(df_video_filter['duration']>0.59)&(df_video_filter['duration']<=5)]

In [ ]:
short_video_cat = short_video.Videocategory.value_counts()[:3]
short_video_cat2018 = short_video[(short_video['year'])==2018]['Videocategory'].value_counts()[:3]
short_video_cat2022 = short_video[(short_video['year'])==2022]['Videocategory'].value_counts()[:3]
short_video_cat2023 = short_video[(short_video['year'])==2023]['Videocategory'].value_counts()[:3]

In [ ]:
print(f'all-time : {short_video_cat}')
print(f'2018 : {short_video_cat2018}')
print(f'2022 : {short_video_cat2022}')
print(f'2023 : {short_video_cat2023}')

### Delta published -View

In [ ]:
then = df_video['Published_date']
now = df_video['time']

time_difference = now - then
time_difference_in_days = time_difference / timedelta(days=1)

df_video['time_difference_in_days'] = time_difference_in_days
df_video.info()

In [ ]:
len(df_video[(df_video.time_difference_in_days)<0])

In [ ]:
df_video_dt = df_video[(df_video.time_difference_in_days)>0]
print(len(df_video))
print(len(df_video_dt))

In [ ]:
df_video_dt.time_difference_in_days.describe()

In [ ]:
df_video_dt.Videocategory.value_counts()[:5]

In [ ]:
plt.figure(figsize=(8,4))
box_plot = sns.boxplot(data=df_video_dt, y='time_difference_in_days', x='Videocategory',fliersize=0,order=['Music', 'People&Blog','Entertainment', 'Film&Animation', 'Comedy'])
plt.xticks(rotation=90)
plt.ylim([-100,4000])

plt.show()

### jour/mois et heure de visionnage

In [ ]:
time_month = pd.DataFrame((df_video['time'].dt.month).value_counts())
time_month = time_month.reset_index()
time_month = time_month.sort_values('time')
time_month

In [ ]:
sns.barplot(data=time_month,x='time',y='count')

In [ ]:
time_week= pd.DataFrame((df_video['time'].dt.weekday).value_counts())
time_week = time_week.reset_index()
time_week= time_week.sort_values('time')


In [ ]:
sns.barplot(data=time_week,x='time',y='count')

In [ ]:
time_hour= pd.DataFrame((df_video['time'].dt.hour).value_counts())

time_hour= time_hour.sort_values('time')
time_hour = time_hour.reset_index()
time_hour

In [ ]:
sns.barplot(data=time_hour,x='time',y='count')

## Categorie analysis

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.kdeplot(data=df_video, x='time',hue='Videocategory',shade=True,palette="Set2")
plt.xlim(df_video['time'].min(),df_video['time'].max())

sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title_fontsize=14)

## Videos statistique

In [ ]:
df_video.info()

In [ ]:
df_video[['Views','Like','Comments']].corr()

In [ ]:
sns.scatterplot(data=df_video,x='Views',y='Like')


In [ ]:
sns.scatterplot(data=df_video,x='Views',y='Comments')


In [ ]:
sns.scatterplot(data=df_video,x='Like',y='Comments')

# Channels stat

In [ ]:
df_channel

In [ ]:
df_channel.sort_values('count',ascending=False)[['Channel_name','count']].iloc[0:5]

In [ ]:
df_channel['per_viewvid'] = 100*(df_channel['count'] / df_channel['Total_videos'])
df_channel['per_viewvid'].describe()

In [ ]:
df_channel.sort_values('per_viewvid',ascending=True).iloc[0:5]

In [ ]:
df_channel['Channel_date'].dt.year.describe()


In [ ]:
df_country = pd.DataFrame(df_channel['country'].value_counts())
df_country = df_country.reset_index()

In [ ]:
plt.pie(df_country['count'],labels=df_country['country'])
plt.show()

In [ ]:
df_channel[['Subscribers','Views','Total_videos']].corr()

In [ ]:
sns.scatterplot(df_channel,x='Subscribers',y='Total_videos')

In [ ]:
df_channel.sort_values('Subscribers',ascending=False).iloc[0]

In [ ]:
df_channel.sort_values('Total_videos',ascending=False).iloc[0]